In [6]:
#pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import time  # 시간을 측정하기 위한 모듈

'''
데이터셋에 대한 설명
기존 MNIST의 숫자 데이터랑 구조는 같지만 더 복잡한 실생활 데이터로 의류 이미지 분류용 데이터이다.

 Fashion-MNIST 데이터셋 개요

샘플 수	       70,000개 이미지
훈련 데이터	   60,000개
테스트 데이터	 10,000개
이미지 크기	   28x28 픽셀 (흑백)
채널 수	       1 (Grayscale)
클래스 수	     10개
형식	         .png로 구성된 28x28 배열을 텐서로 변환해 사용

클래스 목록 (라벨 이름)

클래스번호	라벨이름    	한글 뜻
   0	     T-shirt/top	 티셔츠/상의
   1	     Trouser	     바지
   2	     Pullover	     풀오버 (긴팔 니트류)
   3	     Dress	       원피스
   4	     Coat	         코트
   5	     Sandal	       샌들
   6	     Shirt	       셔츠
   7	     Sneaker	     운동화
   8	     Bag	         가방
   9	     Ankle boot	   발목 부츠


'''
# 1. 데이터 전처리 (이미지 데이터를 텐서로 전환)
transform = transforms.Compose([
    transforms.ToTensor(),   # ToTensor(): 이미지를 텐서로 변환, 0~255 값을 0~1 사이의 값으로 정규화
    transforms.Normalize((0.5,), (0.5,))        # 평균 0.5, 표준편차 0.5로 정규화
])

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# 데이터 미니배치로 나누기
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# 2. MLP 모델 정의
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),               # 다차원 텐서를 1차원으로 펼치는 작업 28x28 → 784
            nn.Linear(784, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)          # 클래스 10개
        )

    def forward(self, x):
        return self.model(x)

model = MLP()

# 3. 손실 함수 정의
criterion = nn.CrossEntropyLoss()

# 4. 옵티마이저 정의 (SGD와 Adam 비교)
sgd_optimizer = optim.SGD(model.parameters(), lr=0.01)
adam_optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Adam의 학습률을 낮춰서 성능 개선

# 5. 학습 함수 정의
def train_model(optimizer, optimizer_name):
    start_time = time.time()

    model.train()
    total_loss = 0
    correct = 0
    total = 0
    for epoch in range(5):  # 5 에폭으로 학습
        epoch_loss = 0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()

        total_loss = epoch_loss / len(train_loader)
        print(f'{optimizer_name} - Epoch [{epoch+1}/5], Loss: {total_loss:.4f}')

    # 학습 시간 측정
    elapsed_time = time.time() - start_time

    # 테스트 정확도 측정
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'{optimizer_name} - Test Accuracy: {accuracy:.2f}%')
    print(f'{optimizer_name} - Training Time: {elapsed_time:.2f} seconds')

    return total_loss, accuracy, elapsed_time

# 6. SGD 학습
print("Training with SGD optimizer:")
sgd_loss, sgd_accuracy, sgd_time = train_model(sgd_optimizer, "SGD")

# 7. Adam 학습
print("\nTraining with Adam optimizer:")
adam_loss, adam_accuracy, adam_time = train_model(adam_optimizer, "Adam")

Training with SGD optimizer:
SGD - Epoch [1/5], Loss: 0.9592
SGD - Epoch [2/5], Loss: 0.5526
SGD - Epoch [3/5], Loss: 0.4868
SGD - Epoch [4/5], Loss: 0.4518
SGD - Epoch [5/5], Loss: 0.4282
SGD - Test Accuracy: 84.17%
SGD - Training Time: 91.95 seconds

Training with Adam optimizer:
Adam - Epoch [1/5], Loss: 0.4072
Adam - Epoch [2/5], Loss: 0.3748
Adam - Epoch [3/5], Loss: 0.3517
Adam - Epoch [4/5], Loss: 0.3314
Adam - Epoch [5/5], Loss: 0.3177
Adam - Test Accuracy: 87.12%
Adam - Training Time: 101.09 seconds
